# Langgraph Bigtool + Long Memory's Similarity Search

## Load MCP Tools

In [ ]:
import json
import os

def load_mcp_servers(config_path):
    """
    Load MCP server definitions from a JSON config file.
    Expects a top-level 'mcpServers' dict in the config.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        config = json.load(f)
    servers = config.get("mcpServers", {})
    # Optionally add default transports if missing
    for name, server in servers.items():
        if "command" in server and "transport" not in server:
            server["transport"] = "stdio"
        if "url" in server and "transport" not in server:
            server["transport"] = "streamable_http"
    return servers

mcp_servers = load_mcp_servers("./mcp_config.json")
mcp_servers

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(mcp_servers)
mcp_tools = await client.get_tools()

print(f"Loaded {len(mcp_tools)} tools.\n")
print(f"Example of a tool:\n{mcp_tools[0]}")

## Define Tool Registry

In [ ]:
import uuid

EXCLUDED_TOOLS = {"search_repositories"}

# Create registry of tools. This is a dict mapping
# identifiers to tool instances.
tool_registry = {
    str(uuid.uuid4()): tool
    for tool in mcp_tools
    if tool.name not in EXCLUDED_TOOLS
}

## Init Long Memory with Tools definitions

In [ ]:
from langchain.embeddings import init_embeddings
from langgraph.store.memory import InMemoryStore

# Index tool names and descriptions in the LangGraph
# Store. Here we use a simple in-memory store.
embeddings = init_embeddings("openai:text-embedding-3-small")

store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,
        "fields": ["description"],
    }
)

for tool_id, tool in tool_registry.items():
    store.put(
        ("tools",),
        tool_id,
        {
            "description": f"{tool.name}: {tool.description}",
        },
    )

store.search(("tools",))

### testing RAG for tools

In [ ]:
# Retrieve relevant tools based on user query
relevant_tools = store.search(
    ("tools",),
    query="I want to create a GitHub issue",
    limit=5
)

for tool in relevant_tools:
    print(f"- {tool}\n")

btw you can also use this technique for Dynamic Tools Selection (previous lesson) instead of LLM categorizations via structured output

## Create Bigtool Agent

In [ ]:
from langgraph_bigtool import create_agent
from langchain.chat_models import init_chat_model
from tools import draw_mermaid_png

# Initialize agent
model = init_chat_model("openai:gpt-4o-mini")

builder = create_agent(model, tool_registry, limit=5)
bigtool_agent = builder.compile(store=store)

draw_mermaid_png(bigtool_agent)

## Testing

In [ ]:
from langchain_core.messages import HumanMessage
from textwrap import dedent

user_query = """
Create a new issue in the langgraph-advanced repository in my account with the following details:

Title: "Add documentation for bigtool example"

Description: 
This issue tracks the need to document the bigtool pattern for handling large numbers of tools.
The documentation should cover:
- How semantic search is used to retrieve relevant tools
- The difference between bigtool and standard tool binding
- Best practices for tool descriptions to improve retrieval accuracy
- Example use cases demonstrating the pattern

This will help users understand when and how to use the bigtool library effectively.
"""


result = await bigtool_agent.ainvoke(
    {"messages": [HumanMessage(content=user_query)]}
)

for message in result['messages']:
    message.pretty_print()

### What the LLM Sees (Tool Schema) and why this is less favorable way

```
{
  "name": "retrieve_tools",
  "description": "Retrieve relevant tools based on a search query.",
  "parameters": {
    "type": "object",
    "properties": {
      "query": {
        "type": "string",
        "description": "Search query"
      }
    },
    "required": ["query"]
  }
}
```

## Reference Links

**1. Handling Large Numbers of Tools in LangGraph**

https://langchain-ai.github.io/langgraph/how-tos/tool-calling/#handle-large-numbers-of-tools

→ Guide to managing agents with many tools using semantic search and filtering strategies to reduce context size and improve tool selection accuracy.

**2. LangGraph BigTool: GitHub Repository**

https://github.com/langchain-ai/langgraph-bigtool

→ Official package for semantic tool retrieval in LangGraph, enabling agents to work with large tool sets by dynamically selecting relevant tools based on query similarity.